# Aggregating runoff from multiple glaciers


To compare runoff projections at basin scale, we need to aggregate the model output over all glaciers that fall within a given basin.  We will do this for the RHONE basin to provide a local example at the Alpine Glaciology Meeting.

31 Jan 2023 | EHU - based on earlier central_europe-aggregation-local

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from cycler import cycler
import datetime 

### Define the case we'll study
We will need to make some consistent selections below: which basin, which SSP, which GCM to compare the two glacier models' output.

In [ ]:
alpine_basins = {'RHINE': '6242',
                 'RHONE': '6243',
                 'PO': '6241'} ## GRDC Major River Basin identifiers for the 3 alpine basins we can study

test_basin = alpine_basins['RHONE'] 

all_ssps = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
which_ssp = all_ssps[2]   #specifying the SSP

#All of the climate models used in GloGEM
modelnames = ['BCC-CSM2-MR','CAMS-CSM1-0','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0']
which_model = modelnames[0] ## specify one model (for GloGEM format)

# PyGEM has 12 GCM forcings as well; TODO: confirm they are in the same order as above.  Until then, use BCC-CSM2-MR
which_gcm = 1 ## applies to PyGEM

In [ ]:
## define local file paths
fstem_pygem = '/Volumes/GoogleDrive/My Drive/Runoff-intercomparison/PyGEM/11/R11_runoff_monthly_c2_ba1_1set_2000_2100-{}-Batch-'.format(which_ssp)
fpath_glogem = '/Volumes/GoogleDrive/My Drive/Runoff-intercomparison/GloGEM-output/RGI11-CentralEurope/files/'  
fpath_glogem_huss = '/Volumes/GoogleDrive/My Drive/Runoff-intercomparison/GloGEM-output/'
fpath_oggm = '/Volumes/GoogleDrive/My Drive/Runoff-intercomparison/OGGM/{}/'.format(test_basin)

### Read in list of glaciers in our basin of interest
Erik Holmgren provided a list of glaciers (by RGI ID) in each major river basin, stored as a JSON file.  Use the function he defined in gha.utils to reference that list and select all the IDs that fall within an example basin.

In [ ]:
import json

def select_glaciers_json(basin='all'):
    '''
    Select glaciers within a basin by MRBID from a json-file,
    which is stored in the data directory.

    Args:
    -----
    basin: str
        String of MRBID or 'all'.

    Returns:
    --------
    If basin is 'all' a list of all relevant glaciers is returned, for
    initiating glacier simulations. If basin is a MRBID the list of glaciers
    within that basin is returned.
    
    Copy of a function written by Erik Holmgren (2022) in holmgren_gha.utils
    '''

    # fpath = './data/rgi_ids_per_basin.json'
    fpath = '/Users/lizz/Documents/Research/Runoff-intercomparison/msc_thesis-multi_gcm/code/data/rgi_ids_per_basin.json' ## correct for local run
    with open(fpath) as f:
        basin_dict = json.load(f)

    if basin.lower() != 'all':
        glacier_list = basin_dict[basin]
    else:
        glacier_list = list(itertools.chain.from_iterable(basin_dict.values()))

    return glacier_list

In [ ]:
gl = select_glaciers_json(test_basin)

## Loop over multiple PyGEM batch files to capture all glaciers of interest

In [ ]:
## use which_ssp defined above for consistency
# fstem_pygem = '/home/ulteelab/Documents/SG/11/R11_runoff_monthly_c2_ba1_1set_2000_2100-{}-Batch-'.format(which_ssp)
batches = ['1-1000.nc', '1001-2000.nc', '2001-3000.nc', '3001-4000.nc']

## loop over them all, drop the irrelevant IDs, and concatenate the result
ds_list = []
for b in batches:
    ds_temp = xr.open_dataset(fstem_pygem+b)
    try:
        ds_list.append(ds_temp.where(ds_temp.RGIId.isin(gl), drop=True))
    except ValueError: ## happens if there are no glaciers from this batch in the selected region
        continue
rhine_ds = xr.concat(ds_list, dim='glacier')

In [ ]:
len(gl)

In [ ]:
single_model_runoff = rhine_ds.glac_runoff_monthly.sel(model=which_gcm).resample(time='A').sum()

In [ ]:
single_model_runoff.sum(dim='glacier').plot()

In [ ]:
scaled_runoff = 1E-9 * single_model_runoff.sum(dim='glacier') ## convert from m3 to km3 (equivalent to Gt water)

fig, ax = plt.subplots()
scaled_runoff.plot()
ax.set(ylabel='Annual runoff [Gt water]', 
       title='Rhone runoff, PyGEM forced by {}, {}'.format(scaled_runoff.Climate_Model.values, which_ssp)
      )

## Old GloGEM output processing to get at single GCM

In [ ]:
## GloGEM -- based on Finn Wimberly script

## Generic filepath to the main data folder
# fpath_glogem = '/home/ulteelab/Documents/Finn/CentralEurope/files/'  

#Read discharge (monthly, starting Oct '79 for 1980 hydro year) to a pandas dataframe
dfdischarge1 = pd.read_csv(fpath_glogem +'{}/{}/centraleurope_Discharge_gl_r1.dat'.format(which_model, which_ssp), sep='\s+', header=None, skiprows=1, index_col=0)

#Read glacier area (INITIAL) to a pandas dataframe
df_area_huss = pd.read_csv(fpath_glogem_huss+'/Area-20230124/{}/RGIreg11_Area_individual.dat'.format(which_ssp), 
                           sep='\s+', header=0, skiprows=1, index_col=0)
df_area_huss.columns = pd.to_datetime(df_area_huss.columns)
df_area_init = df_area_huss.loc[:,'1980-01-01'] ## select just the initial area to compute basin runoff


In [ ]:
#Reindexing discharge columns w/ year-month format
new_dischargeindices=[]
for y in range(1980,2101):
    for m in range (1,13):
        this_index = '{}-{}'.format(y,m)
        new_dischargeindices.append(this_index)
dfdischarge1.columns = pd.to_datetime(new_dischargeindices)


In [ ]:
dfrunoff = dfarea1 * dfdischarge1
# dfrunoff.columns = dfrunoff.transpose().index.shift(periods=-3, freq='MS') ## shift times to align with hydro year
## DON'T SHIFT when using annual resampling -- return to this later for seasonal analysis
dfrunoff

### Composite to example basin
GloGEM output is stored differently from PyGEM output, so this selection will be different.  We still want to use the list of RGI IDs `gl` defined above.

Set up a new list, trimming the full RGI ID to only the integer values included in GloGEM index:

In [ ]:
# RGI_stem ='RGI60-11.00000'

new_list = [int(s.replace('RGI60-11.0', '')) for s in gl] ## trim leading characters to search this df
new_list_trimmed = [f for f in new_list if f in dfdischarge1.index]

In [ ]:
dfglogem_masked = dfdischarge1.replace(to_replace=-99., value=None)

In [ ]:
rhone_glogem = dfglogem_masked.loc[new_list_trimmed] ## select the rows with IDs found in the list ## select the rows with IDs found in the list
# rhone_glogem
annual_rhone = rhone_glogem.transpose().resample('AS').sum() ## annual total runoff per glacier

In [ ]:
annual_rhone.transpose().sum().plot() ## sum annual values over all glaciers

In [ ]:
area_index_list = [float(s.replace('RGI60-', '')) for s in gl
                  if int(s.replace('RGI60-11.0', '')) in dfdischarge1.index] 
## area df and discharge df have different labelling conventions
## trim to make sure they cover the same glaciers

df_area_rhone = df_area_init.loc[area_index_list]

In [ ]:
new_labels = [float('11.0'+str(s)) for s in annual_rhone.columns]
new_labels
annual_rhone.columns = new_labels ## remember that annual_rhone is transposed from the discharge

In [ ]:
annual_rhone

In [ ]:
rhone_runoff = annual_rhone.mul(df_area_rhone, axis=1)

In [ ]:
rhone_runoff

In [ ]:
glogem_runoff_gt = 1e-3 * rhone_runoff.transpose().sum() ## re-scale for a conversion issue (m * km2 -> km3)

In [ ]:
fig, ax = plt.subplots()
glogem_runoff_gt.plot(ax=ax)
ax.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1))) ## visualize runoff to the example basin from GloGEM

Plot this side by side with PyGEM:

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, sharey=True)
glogem_runoff_gt.plot(ax=ax1, label='GloGEM')
scaled_runoff.plot(ax=ax2, label='PyGEM')
ax1.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
        title='GloGEM'
       )
ax2.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='PyGEM'
       )
fig.suptitle('Rhone runoff forced by {}, {} \n {}'.format(
    scaled_runoff.Climate_Model.values, which_ssp, datetime.date.today()))

### Add OGGM output
Pull in data from Erik Holmgren's simulations to compare.


In [ ]:
fname = fpath_oggm + 'oggm_compiled_{}_{}_{}.nc'.format(test_basin, which_model, which_ssp)

In [ ]:
ds_og = xr.open_dataset(fname, engine='h5netcdf')

In [ ]:
total_runoff_disagg = ds_og.melt_off_glacier + ds_og.melt_on_glacier + ds_og.liq_prcp_off_glacier + ds_og.liq_prcp_on_glacier
total_runoff = 1e-12 * total_runoff_disagg.sum(dim='rgi_id')[:-1] ## numbers are in kg, so multiply by 1e-12 to get Gt

In [ ]:
fig, ax = plt.subplots()
total_runoff.plot()
ax.set(ylabel='Annual runoff [Gt]', xlabel='Year',
      title='Rhone basin runoff simulated with OGGM, forced by {}, {} \n {}'.format(
      which_model, which_ssp, datetime.date.today()))

Plot all three models side by side:

In [ ]:
t = pd.to_datetime(ds_og.indexes['time'].values, format='%Y') ## modify the OGGM indices to make plot behave

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3, sharey=True, figsize=(12,5))
glogem_runoff_gt.plot(ax=ax1, label='GloGEM')
scaled_runoff.plot(ax=ax2, label='PyGEM')
ax3.plot(t[:-1], total_runoff, label='OGGM')
ax1.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
        title='GloGEM'
       )
ax2.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='PyGEM'
       )
ax3.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='OGGM'
       )
fig.suptitle('Rhone runoff forced by {}, {} \n {} \n'.format(
    scaled_runoff.Climate_Model.values, which_ssp, datetime.date.today()))

Running means?

In [ ]:
rolling_yrs = 15

fig, (ax1,ax2,ax3) = plt.subplots(1,3, sharey=True, figsize=(12,5))
glogem_runoff_gt.rolling(window=rolling_yrs).mean().plot(ax=ax1, label='GloGEM')
scaled_runoff.rolling(time=rolling_yrs, min_periods=10).mean().plot(ax=ax2, label='PyGEM')
ax3.plot(t[:-1], total_runoff.rolling(time=rolling_yrs, min_periods=10).mean(), label='OGGM')
ax1.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
        title='GloGEM'
       )
ax2.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='PyGEM'
       )
ax3.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='OGGM'
       )
fig.suptitle('Rhone runoff forced by {}, {}, {} year rolling mean  \n {} \n'.format(
    scaled_runoff.Climate_Model.values, which_ssp, rolling_yrs, datetime.date.today()))

And rolling var:

In [ ]:
rolling_yrs = 15

fig, (ax1,ax2,ax3) = plt.subplots(1,3, sharey=True, figsize=(12,5))
glogem_runoff_gt.rolling(window=rolling_yrs).var().plot(ax=ax1, label='GloGEM')
scaled_runoff.rolling(time=rolling_yrs, min_periods=10).var().plot(ax=ax2, label='PyGEM')
ax3.plot(t[:-1], total_runoff.rolling(time=rolling_yrs, min_periods=10).var(), label='OGGM')
ax1.set(xlabel = 'Year', ylabel='Annual runoff variance',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
        title='GloGEM'
       )
ax2.set(xlabel = 'Year', ylabel='Annual runoff variance',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='PyGEM'
       )
ax3.set(xlabel = 'Year', ylabel='Annual runoff variance',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='OGGM'
       )
fig.suptitle('Rhone runoff variance forced by {}, {}, {} year rolling  \n {} \n'.format(
    scaled_runoff.Climate_Model.values, which_ssp, rolling_yrs, datetime.date.today()))

Plot series faintly behind the rolling mean

In [ ]:
rolling_yrs = 15
line_color='darkblue'

fig, (ax1,ax2,ax3) = plt.subplots(1,3, sharey=True, figsize=(12,5))

glogem_runoff_gt.plot(ax=ax1, label='GloGEM', lw=0.5, alpha=0.5, color=line_color)
glogem_runoff_gt.rolling(window=rolling_yrs).mean().plot(ax=ax1, label='GloGEM rolling mean', color=line_color)

scaled_runoff.plot(ax=ax2, label='PyGEM', lw=0.5, alpha=0.5, color=line_color)
scaled_runoff.rolling(time=rolling_yrs, min_periods=10).mean().plot(ax=ax2, label='PyGEM rolling mean', color=line_color)

ax3.plot(t[:-1], total_runoff, label='OGGM', lw=0.5, alpha=0.5, color=line_color)
ax3.plot(t[:-1], total_runoff.rolling(time=rolling_yrs, min_periods=10).mean(), label='OGGM rolling mean', color=line_color)

ax1.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
        title='GloGEM'
       )
ax2.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='PyGEM'
       )
ax3.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='OGGM'
       )
# fig.suptitle('Rhone runoff forced by {}, {} \n {} \n'.format(
#     scaled_runoff.Climate_Model.values, which_ssp, datetime.date.today()))
fig.suptitle('Rhone runoff forced by {}, {} \n'.format(
    scaled_runoff.Climate_Model.values, which_ssp))

In [ ]:
rolling_yrs = 15
line_color='darkblue'


fig, (ax1,ax2,ax3) = plt.subplots(1,3, sharey=True, figsize=(12,5))

# ax1.plot(glogem_runoff_gt, lw=0.5, alpha=0.5, color=line_color)
ax1.plot(glogem_runoff_gt.rolling(window=rolling_yrs, center=True).var(), color=line_color)
# glogem_runoff_gt.rolling(window=rolling_yrs).var().plot(ax=ax1, label='GloGEM', color=line_color)
scaled_runoff.rolling(time=rolling_yrs, min_periods=10, center=True).var().plot(ax=ax2, label='PyGEM', color=line_color)
ax3.plot(t[:-1], total_runoff.rolling(time=rolling_yrs, min_periods=10, center=True).var(), label='OGGM', color=line_color)
ax1.set(xlabel = 'Year', ylabel='Annual runoff variance',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
        title='GloGEM'
       )
ax2.set(xlabel = 'Year', ylabel='',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='PyGEM'
       )
ax3.set(xlabel = 'Year',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='OGGM'
       )

for ax in (ax1,ax2,ax3):
    ax.xaxis.set_major_locator(mdates.YearLocator(100, month=1, day=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

plt.gcf().autofmt_xdate()

fig.suptitle('Rhone runoff variance forced by {}, {}, {} year rolling \n'.format(
    scaled_runoff.Climate_Model.values, which_ssp, rolling_yrs))

In [ ]:
## add labels for AGM poster
import matplotlib.dates as mdates
rolling_yrs = 15
line_color='darkblue'

fig, (ax1,ax2,ax3) = plt.subplots(1,3, sharey=True, figsize=(12,5))

## GloGEM
# ax1.xaxis.set_major_locator(mdates.YearLocator(20, month=1, day=1))
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
# rhine_runoff_total_gt.plot(ax=ax1, lw=0.5, alpha=0.5, color=line_color)
# rhine_runoff_total_gt.rolling(window=rolling_yrs).mean().plot(ax=ax1, color=line_color)
ax1.plot(glogem_runoff_gt, lw=0.5, alpha=0.5, color=line_color)
ax1.plot(glogem_runoff_gt.rolling(window=rolling_yrs, center=True).mean(), color=line_color)

## PyGEM
scaled_runoff.plot(ax=ax2, lw=0.5, alpha=0.5, color=line_color)
scaled_runoff.rolling(time=rolling_yrs, min_periods=10, center=True).mean().plot(ax=ax2, color=line_color)

## OGGM
ax3.plot(t[:-1], total_runoff, label='Annual', lw=0.5, alpha=0.5, color=line_color)
ax3.plot(t[:-1], total_runoff.rolling(time=rolling_yrs, min_periods=10, center=True).mean(), label='15-yr rolling mean', color=line_color)
ax3.legend(loc='lower right') 

ax1.set(xlabel = 'Year', ylabel='Annual runoff [Gt water]',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
        title='GloGEM'
       )
ax2.set(xlabel = 'Year', ylabel='',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='PyGEM'
       )
ax3.set(xlabel = 'Year',
      xlim=(datetime.date(2000,1,1), datetime.date(2101,1,1)),
      title='OGGM'
       )

for ax in (ax1,ax2,ax3):
    ax.xaxis.set_major_locator(mdates.YearLocator(20, month=1, day=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

plt.gcf().autofmt_xdate()
# fig.suptitle('Rhone runoff forced by {}, {} \n {} \n'.format(
#     scaled_runoff.Climate_Model.values, which_ssp, datetime.date.today()))
fig.suptitle('Rhone runoff forced by {}, {} \n'.format(
    scaled_runoff.Climate_Model.values, which_ssp))

## Okay, show us RCP 4.5 to go along with the SPEI plot